In [3]:
from random import seed
from random import randrange
from csv import reader
from math import sqrt

In [4]:
#load csv
def load_csv(filename):
    dataset = list()
    with open(filename,'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

In [5]:
#k-fold cv split
def cross_validation_split(dataset,n_folds):
    dataset_split=list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset)/n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold)<fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split


In [22]:
def split_data(dataset,ratio):
    #train=list()
    test = list()
    dataset_copy = list(dataset)
    test_size = int(len(dataset)*ratio)
    while len(test)<test_size:
        index = randrange(len(dataset_copy))
        test.append(dataset_copy.pop(index))
    return dataset_copy,test

In [7]:
#calculate accuracy percentage
def accuracy_metric(actual,predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct+=1
    return correct/float(len(actual))*100.0

In [8]:
def recall_metric(actual,predicted):
    correct = 0
    positive = 0
    #print(actual)
    for i in range(len(actual)):
        if actual[i] == '1':
            positive+=1
            if actual[i]==predicted[i]:
                correct+=1
    if positive==0:
        return 0
    else:
        return correct/positive*100.0

In [23]:
#evaluate, using cross validation split
def evaluate_algorithm(dataset,algorithm,ratio,*args):
    #print(algorithm)
    #folds = cross_validation_split(dataset,n_folds)
    train,test = split_data(dataset,ratio)
    test_copy = list()
    for row in test:
        row_copy=list(row)
        row_copy[-1]=None
        test_copy.append(row_copy)
    predicted = algorithm(train,test_copy,*args)
    actual = [row[-1] for row in test]
    accuracy = accuracy_metric(actual,predicted)
    recall = recall_metric(actual,predicted)
    return accuracy,recall
    '''
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set,[])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None   #here might have some problems, but it truely works
        predicted = algorithm(train_set,test_set,*args)
        actual = [row[-1] for row in fold]
        #print(actual)
        accuracy = accuracy_metric(actual,predicted)
        recall = recall_metric(actual,predicted)
        scores.append(accuracy)
        recalls.append(recall)
    return max(scores),max(recalls)
    '''

In [10]:
# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

In [11]:
#calculate gini
def gini_index(groups,classes):
    #count all samples at split point
    n_instances = float(sum([len(group) for group in groups])) #total
    #sum weighted gini index for each group
    gini = 0.0
    for group in groups:
        size = float(len(group))
        if size == 0 : #avoid divide by zero
            continue
        score = 0.0
        #score the group based on the score for each class
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val)/size
            score+=p*p
        #weight the group score by its relative size
        gini += (1.0-score)*(size/n_instances)
    return gini

In [12]:
#select the best split point for a dataset
def get_split(dataset,n_features):
    class_values = list(set(row[-1] for row in dataset))
    b_index,b_value,b_score,b_groups=999,999,999,None
    features = list()
    while len(features)<n_features:
        index = randrange(len(dataset[1])-1)
        if index not in features:
            features.append(index)
    for index in features:
        for row in dataset:
            groups = test_split(index,row[index],dataset)
            gini = gini_index(groups,class_values)
            if gini<b_score:
                b_index,b_value,b_score,b_groups = index,row[index],gini,groups
    #print(b_index,b_value,b_groups)
    return {'index':b_index,'value':b_value,'groups':b_groups}

In [13]:
#create a terminal node value
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes),key=outcomes.count)

In [14]:
#create child splits for a node or make terminal
def split(node,max_depth,min_size,n_features,depth):
    left,right = node['groups']
    del(node['groups'])
    #check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left+right)
        return
    #check for max depth
    if depth>=max_depth:
        node['left'] = node['right'] = to_terminal(left),to_terminal(right)
        return
    #process left child
    if len(left)<=min_size:
        node['left']= to_terminal(left)
    else:
        node['left'] = get_split(left,n_features)
        split(node['left'],max_depth,min_size,n_features,depth+1)#recursion
    #process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right, n_features)
        split(node['right'], max_depth, min_size, n_features, depth+1)

In [15]:
# Build a decision tree
def build_tree(train, max_depth, min_size, n_features):
    root = get_split(train, n_features)
    split(root, max_depth, min_size, n_features, 1)
    return root

In [16]:
# Make a prediction with a decision tree
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

In [17]:
# Create a random subsample from the dataset with replacement
#bootstrapping
def subsample(dataset, ratio):
    sample = list()
    n_sample = round(len(dataset) * ratio)
    while len(sample) < n_sample:
        index = randrange(len(dataset))
        sample.append(dataset[index])
    return sample

In [18]:
# Make a prediction with a list of bagged trees
def bagging_predict(trees, row):
    predictions = [predict(tree, row) for tree in trees]
    return max(set(predictions), key=predictions.count)

In [19]:
# Random Forest Algorithm
def random_forest(train, test, max_depth, min_size, sample_size, n_trees, n_features):
    trees = list()
    for _ in range(n_trees):
        sample = subsample(train, sample_size)
        tree = build_tree(sample, max_depth, min_size, n_features)
        trees.append(tree)
        #print(tree)
    predictions = [bagging_predict(trees, row) for row in test]
    return(predictions)

In [44]:
seed(2)
dataset = load_csv('testfile.csv')
n_folds = 10
max_depth = 9
min_size = 1
sample_size = 1.0
n_features = 7
ratio = 0.2

In [37]:
for n_trees in [6]:
    score,recall = evaluate_algorithm(dataset,random_forest,ratio, max_depth, min_size,sample_size, n_trees, n_features)
    print('Trees: %d' % n_trees)
    print('Score: %.3f%%' % score)
    print('Recall: %.3f%%' % recall)
    #print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Trees: 6
Score: 50.806%
Recall: 37.500%


In [38]:
for n_trees in [10]:
    score,recall = evaluate_algorithm(dataset,random_forest,ratio, max_depth, min_size,sample_size, n_trees, n_features)
    print('Trees: %d' % n_trees)
    print('Score: %.3f%%' % score)
    print('Recall: %.3f%%' % recall)
    #print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Trees: 10
Score: 65.323%
Recall: 55.385%


In [46]:
for n_trees in [19]:
    score,recall = evaluate_algorithm(dataset,random_forest,ratio, max_depth, min_size,sample_size, n_trees, n_features)
    print('Trees: %d' % n_trees)
    print('Score: %.3f%%' % score)
    print('Recall: %.3f%%' % recall)
    #print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Trees: 19
Score: 62.903%
Recall: 55.652%
